In [2]:
import django
django.setup()
from tree.models import Tree, Node
import os
from django.contrib.auth.models import User
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
#python manage.py shell_plus --notebook

In [ ]:
def findChildren(number):
    if number == 1:
        return [2,3]
    if number %2 == 0:
        return[number *2 +2, number *2, ]
    if number % 2 ==1:
        return [number*2 -1, number *2 +1]

def findParent(Node):
    #returns plain number
    if Node %2 == 0:
        parent = int(Node / 2)
        if not parent % 2 == 0 and not parent == 1:
            parent -= 1
    else:
        if Node == 1:
            return None
        parent = int(Node /2)
        if parent % 2 == 0:
            parent += 1
    return parent




def getRow(startNode, rowNumber):
    rowNumber  = rowNumber 
    
    if startNode == 1:
        limits = [2, 3]
        centerTwo = [2,3]
        while rowNumber >0:
            limits = [limits[0] *2 +2, limits[1] * 2 +1]
            rowNumber -=1

            center = int(limits[0] /2)
            centerTwo =[center +1, center +2]   
        return limits, centerTwo

    if startNode % 2 == 1 and startNode != 1:        
        limits = [startNode * 2 -1, startNode * 2 +1]
        while rowNumber >0:
            limits = [limits[0] *2 -1, limits[1] * 2 +1]
            rowNumber -=1
        center = int(limits[0] + (limits [1] - limits[0])/2)
        centerTwo =[center -1, center +1]   
        return limits, centerTwo

    if startNode % 2 == 0:
        limits = [startNode * 2 +2, startNode * 2 ]
        while rowNumber >0:
            limits = [limits[0] *2 +2, limits[1] * 2]
            rowNumber -=1
        center = int((limits[0] - limits[1] )/2)
        centerTwo =[limits[1]+ center +1, limits[1] + center -1]   
        return limits, centerTwo
def getFreeNode(user = None):
    suggestedNode = 1;
    while True:
        if not suggestedNode in tree:
            return suggestedNode
        else:
            suggestedNode += 1

def nodeGenerator(startNode = 1):
    row = 0
    yield startNode   
        
    if startNode ==1:
        while True:
            limits, centerTwo = getRow(startNode,row)
            smallerNode = centerTwo[0]
            largerNode = centerTwo[1]
            while smallerNode <= limits[0]:                
                yield smallerNode                  
                              
                yield largerNode                   

                smallerNode +=2
                largerNode += 2
            row += 1

            limits, centerTwo = getRow(startNode,row)        
        
    
    if startNode % 2 == 1 and not startNode == 1:    
        while True:
            limits, centerTwo = getRow(startNode,row)
            smallerNode = centerTwo[0]
            largerNode = centerTwo[1]
            while smallerNode >= limits[0]:             
                
                                   
                yield smallerNode                                   
                              
                yield largerNode                 


                smallerNode -=2
                largerNode += 2
            row += 1

            limits, centerTwo = getRow(startNode,row)
        
            
    if startNode % 2 == 0:    
        while True:
            limits, centerTwo = getRow(startNode,row)
            
            upperNode = centerTwo[0] 
            lowerNode = centerTwo[1] 
            while lowerNode >= limits[1]:               
                                
                yield upperNode                
                                 
                yield lowerNode                    

                upperNode +=2
                lowerNode -= 2
            row += 1
            limits, centerTwo = getRow(startNode,row)

def parentGenerator(Node):
    
    while True:
        if Node ==1:            
            break
            
        if Node %2 == 0:
            parent = int(Node / 2)
            if not parent % 2 == 0 and not parent == 1:
                parent -= 1
                
            yield {"parent":parent, "child":Node}
            Node = parent
            
        if Node %2 == 1 and not Node == 1:
            parent = int(Node /2)
            if parent % 2 == 0:
                parent += 1
            yield {"parent":parent, "child":Node}
            Node = parent

def childOf(parent, child):
    if parent == child:
        return True
    generator = parentGenerator(child)
    for gen_parent in generator:
        if gen_parent ["parent"] == parent:
            
            return True
    return False

def identifyParent(node, *args):
    #which argument is the nodes parent
    for parent in args:
        if parent == node:
            return parent
    generator = parentGenerator(node)
    while True:
        node = next(generator)        
        for parent in args:
            if parent == node["parent"]:
                return parent

In [226]:
user = User.objects.first()
tree = Tree.objects.first()
nodes = tree.node_set.all().order_by("-number")
highestRow = getRow(nodes.first().number).get("rowNumber")

In [227]:
def getRow(nodeNumber):
    if nodeNumber == 1:
        return {"rowNumber":0, "limits": {"even_limits": None, "odd_limits": None}}
    rowNumber  = 1
    limits = [2, 3]
    nodes_in_row = 2
    even_limits = [2,2]
    odd_limits = [3,3]
    while not (nodeNumber >limits[0]/2 and nodeNumber <= limits[1]):
        limits = [limits[0] *2 +2, limits[1] * 2 +1]
        even_limits = [int(limits [0]/2) + 1, limits[0]]
        odd_limits = [int(limits[0]/2) +2, limits[1]]       
        rowNumber +=1
        nodes_in_row = int(limits[1]- limits[0]/2)
    #return rowNumber, [even_limits, odd_limits], 
    row = {"rowNumber":rowNumber, "limits": {"even_limits": even_limits, "odd_limits": odd_limits}}
    return row

def getX(nodeNumber):
    if nodeNumber == 1:
        return 0.5
    row = getRow(nodeNumber)
    
    if nodeNumber % 2 == 1:
        #all even numbers are considered smaller
        smaller_even_numbers = row["limits"]["even_limits"][0] / 2
        smaller_odd_numbers = ( nodeNumber - row["limits"]["odd_limits"][0] )/2
    else:
        smaller_even_numbers = (row["limits"]["even_limits"][1] - nodeNumber)/2
        smaller_odd_numbers = 0 
    

        
  
    smaller_numbers_sum = smaller_odd_numbers + smaller_even_numbers        
    nodes_sum = row["limits"]["even_limits"][0]
    fairCut = 1/ (nodes_sum + 2)
    #fraction of all the smaller numbers which counts in the two borders as well
    location = fairCut* (smaller_numbers_sum + 1) + fairCut/2
    return round(location, 5)   
    

def getY(row, rowCount):
    fairCut = 1/rowCount
    return row * fairCut
    

In [239]:
getY(0,6)

0.0

In [242]:
from tree.serializers import NodeSerializer
serializer = NodeSerializer(nodes, many=True)  
   
for node in serializer.data:
    node["x"] = getX(node["number"])
    nodeRow = getRow(node["number"]).get("rowNumber")
    node["y"] = getY(nodeRow, highestRow)
    
  
